# Creating a FSAF model using Resnet-50

We first install the FSAF library

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
!git clone https://github.com/joheras/FSAF

In [0]:
cd FSAF/

In [0]:
!pip install -r requirements.txt

We load the necessary code and start the training process.

We download the dataset. If you are working with Google Colab, you have several options to download the dataset in this notebook, see the available options in the LabelDetection documentation.

In [0]:
!wget https://www.dropbox.com/s/48i41x9wmkohb83/datasets-fruit-news.zip?dl=0 -O datasets.zip

In [0]:
!unzip datasets.zip

We train the model.


In [0]:
!python3 train.py --backbone resnet50 --gpu 0 --batch-size 4 --epochs 25 --steps 32 pascalCustom datasets/VOCdataset

The model was evaluated at the end of the previous execution.

In [0]:
!mv snapshots/*/resnet50_pascalCustom_25.h5 output.h5

In [0]:
!rm -rf snapshots/*

At the end you will have a file called output.h5 that together with the names file can be included in the application to be employed with new images. 

----------------------

# Data distillation

We first install the library for ensemble methods.

In [0]:
!git clone --recurse-submodules https://github.com/ancasag/ensembleObjectDetection.git

In [0]:
cd ensembleObjectDetection/TestTimeAugmentation

We install additional libraries that are required.

In [0]:
!pip install clodsa
!pip install mrcnn
!pip install keras-retinanet
!pip install gluoncv
!pip install mxnet

The following cells apply data distillation.

In [0]:
import testTimeAugmentation
import function
import os
import shutil
import argparse
import ensembleOptions
from mainTTA import tta
from imutils import paths

In [0]:
pathImg = '/content/FSAF/datasets/unlabelled/'

In [0]:
fsaf = testTimeAugmentation.FSAF('/content/FSAF/output.h5', '/content/FSAF/datasets/VOCdataset/classes.csv')

In [0]:
myTechniques = [ "histo","hflip","none"]

In [0]:
option = "affirmative"

In [0]:
tta(fsaf,myTechniques,pathImg,option)

Now, we reorganize the dataset.

In [0]:
!ls /content/FSAF/datasets/unlabelled/*.jpg >> /content/FSAF/datasets/VOCdataset/ImageSets/Main/train.txt

In [0]:
!sed -i 's#/content/FSAF/datasets/unlabelled/##g' /content/FSAF/datasets/VOCdataset/ImageSets/Main/train.txt

In [0]:
!sed -i 's#.jpg##g' /content/FSAF/datasets/VOCdataset/ImageSets/Main/train.txt

In [0]:
!mv /content/FSAF/datasets/unlabelled/*.jpg /content/FSAF/datasets/VOCdataset/JPEGImages/

In [0]:
!mv /content/FSAF/datasets/salida/output/*.xml /content/FSAF/datasets/VOCdataset/Annotations/

In [0]:
cd /content/FSAF/

Finally, we retrain the model and evaluate its performance. 


In [0]:
!python3 train.py --backbone resnet50 --gpu 0 --batch-size 4 --epochs 25 --steps 32 pascalCustom datasets/VOCdataset

In [0]:
!mv snapshots/*/resnet50_pascalCustom_25.h5 output.h5

---------------------------------

# Using the model in LabelDetection

If you want to use the trained model with LabelDetection, you must download the following files:

- FSAF/output.h5
- FSAF/datasets/classes.names